# Tugas Eksplorasi Pemrograman 1
Mata Kuliah: Perolehan Informasi

Author: Adrianus Saga Ekakristi, Gibran Brahmanta

Dalam tugas ini, anda akan mempelajari *text preprocessing* yang umum dilakukan sebagai langkah awal dalam sistem Information Retrieval. Terdapat 4 proses utama yang akan dibahas, yaitu *tokenization*, *lemmatization*, *stemming*, dan *stop words removal*.

Notebook ini terdiri dari 2 bagian. Pertama, anda akan diberikan contoh code *text preprocessing* yang diterapkan untuk data dalam bahasa Indonesia. Pada bagian kedua, anda akan diminta untuk menulis code untuk melakukan pemrosesan serupa untuk data dalam bahasa Inggris.

Selamat mengerjakan dan semoga bermanfaat!

## Bagian 1: Text Preprocessing untuk Bahasa Indonesia

Bagian ini akan membahas mengenai contoh pemrosesan teks bahasa Indonesia. Silahkan pelajari dan coba pahami code-code dibawah ini.

In [2]:
import time
start_time_id: int = time.time()
! date

^C
The current date is: 12/09/2023 
Enter the new date: (dd-mm-yy) 


### Preparasi, Instalasi Packages, Download Resources

Beberapa packages perlu diinstall terlebih dahulu, seperti:
- dataset (Hugginface Dataset)
- PySastrasi (Implementasi Python untuk Sastrawi)
- package-package lainnya sesuai kebutuhan

In [1]:
!pip install datasets==2.10.1 PySastrawi==1.2.0 stanza==1.5.0

from typing import List, Dict, Any, Set, Tuple
from tqdm import tqdm

import json
import multiprocessing
import re

import stanza
stanza.download('id', processors = 'tokenize,mwt,pos,lemma')

Defaulting to user installation because normal site-packages is not writeable


2023-09-12 21:18:32 INFO: Downloading these customized packages for language: id (Indonesian)...
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| pretrain  | conll17 |

2023-09-12 21:18:32 INFO: File exists: C:\Users\USER\stanza_resources\id\tokenize\gsd.pt
2023-09-12 21:18:32 INFO: File exists: C:\Users\USER\stanza_resources\id\mwt\gsd.pt
2023-09-12 21:18:32 INFO: File exists: C:\Users\USER\stanza_resources\id\pos\gsd.pt
2023-09-12 21:18:32 INFO: File exists: C:\Users\USER\stanza_resources\id\lemma\gsd.pt
2023-09-12 21:18:33 INFO: File exists: C:\Users\USER\stanza_resources\id\pretrain\conll17.pt
2023-09-12 21:18:33 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.


### Load Data

Untuk contoh ini, akan digunakan data corpus Indonesian mC4 (Colossal Clean Crawled Corpus, Raffel et al., 2019), yaitu kumpulan hasil crawling web oleh Common Crawl yang sudah dibersihkan oleh Raffel et al., 2019 serta Allen AI, dan kemudian dilakukan filter untuk bahasa Indonesia.

Load data dilakukan dengan memanfaatkan library HuggingFace Dataset. Beberapa parameter penting:
- path = 'indonesian-nlp/mc4-id': merujuk ke https://huggingface.co/datasets/indonesian-nlp/mc4-id
- name = 'tiny': config untuk melakukan loading data dalam jumlah tertentu (tiny)
- split = 'X[:N%]': ambil N% split X data dari dataset yang telah diunduh (e.g. train[:5%])
- num_proc: melakukan penarikan data secara parallel dengan jumlah process sebanyak nilai yang digunakan dalam parameternya

Referensi:
- https://huggingface.co/docs/datasets/loading
- https://huggingface.co/docs/datasets/main/en/package_reference/loading_methods#datasets.load_dataset

In [3]:
from datasets import load_dataset, Dataset, DatasetDict
dataset_id: Dataset = load_dataset(
    'indonesian-nlp/mc4-id',
    'tiny',
    split = 'validation[:5%]',
    num_proc = min(8 * multiprocessing.cpu_count(), 60)
)

Found cached dataset mc4-id (C:/Users/USER/.cache/huggingface/datasets/indonesian-nlp___mc4-id/tiny/1.0.0/721475d1d1a512521de99189896104190f5ef3c5dbbdd511442097b116178061)


Setelah data selesai ditarik sebagai variable `dataset_id`, kita dapat melihat lebih detail dataset ini beserta dengan sample salah satu row.

In [4]:
print(f"dataset variable data type: {type(dataset_id)}")
print(f"dataset: {dataset_id}")
print(f"dataset sample: {json.dumps(dataset_id[0], indent = 2)}")

dataset variable data type: <class 'datasets.arrow_dataset.Dataset'>
dataset: Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 803
})
dataset sample: {
  "text": "03 Desember 2018, 11: 55: 59 WIB | editor : Perdana\nPERAWATAN naskah kuno berusia ratusan tahun koleksi Rekso Pustoko memerlukan biaya tak sedikit biaya. Dirinya tak menyangkal bahwa perawatan untuk buku-buku berusia sekitar 200 tahun itu terbilang kurang. Besarnya biaya perawatan masih menjadi alasan utama sebagian besar koleksi belum tersentuh perawatan layaknya naskah dan manuskrip kuno di museum-museum atau perpustakaan besar lainnya.\nSalah satu perawatan yang masih belum baik adalah fumigasi. Hingga saat ini, perawatan untuk membunuh biota yang merusak arsip hanya bisa dilakukan setahun sekali untuk satu ruang. Idealnya fumigasi dilakukan dua kali dalam setahun.\nSelain fumigasi, untuk menjaga ketahanan kertas agar tidak dimakan ngengat bisa dilakukan dengan melakukan enkapsulasi dan alih media. Biay

Dapat dilihat bahwa terdapat beberapa kolom/field (title, text, domain, dsb) dalam dataset ini. Dalam contoh ini, kita hanya memerlukan kolom text. Untuk membuang kolom lain, dapat digunakan function remove_columns.

In [5]:
dataset_id = dataset_id.remove_columns(["timestamp", "url"])
print(f"dataset: {dataset_id}")

dataset: Dataset({
    features: ['text'],
    num_rows: 803
})


Untuk keperluan ilustrasi input dan output dari masing-masing langkah nantinya, kita dapat mengambil salah satu konten artikel dari dataset.

In [6]:
example_passage_id: str = dataset_id[0]['text']
print(f"example passage: {example_passage_id}")

example passage: 03 Desember 2018, 11: 55: 59 WIB | editor : Perdana
PERAWATAN naskah kuno berusia ratusan tahun koleksi Rekso Pustoko memerlukan biaya tak sedikit biaya. Dirinya tak menyangkal bahwa perawatan untuk buku-buku berusia sekitar 200 tahun itu terbilang kurang. Besarnya biaya perawatan masih menjadi alasan utama sebagian besar koleksi belum tersentuh perawatan layaknya naskah dan manuskrip kuno di museum-museum atau perpustakaan besar lainnya.
Salah satu perawatan yang masih belum baik adalah fumigasi. Hingga saat ini, perawatan untuk membunuh biota yang merusak arsip hanya bisa dilakukan setahun sekali untuk satu ruang. Idealnya fumigasi dilakukan dua kali dalam setahun.
Selain fumigasi, untuk menjaga ketahanan kertas agar tidak dimakan ngengat bisa dilakukan dengan melakukan enkapsulasi dan alih media. Biaya enkapsulasi sendiri cukup besar. Satu lembar enkapsulasi dihargai sebesar Rp 20 ribu. Dalam sebulan ekapsulasi bisa dilakukan pada 1.000 lembar manuskrip sehingga bia

### 1.1: Tokenization

Tokenisasi merupakan proses mengubah teks (input) menjadi serangkaian token (output). Token adalah komponen terkecil dari teks, seperti kata, tanda baca, dan angka, dan berbagai variasi lainnya.

Terdapat berbagai cara untuk melakukan tokenisasi, mulai dari penggunaan spasi sebagai delimiter, regular expression (regex), hingga metode berbasis machine learning. Dalam contoh ini, kita akan menggunakan regex untuk melakukan tokenisasi.

Referensi:
- https://docs.python.org/3/howto/regex.html

In [7]:
# Tokenize
tokenizer_pattern: str = r'\w+'
def tokenize_text_id(text: str, tokenizer_pattern: str) -> List[str]:
  tokens: List[str] = re.findall(tokenizer_pattern, text)
  return tokens

example_tokens: List[str] = tokenize_text_id(
  text = example_passage_id,
  tokenizer_pattern = tokenizer_pattern,
)
print(f"example of tokenized text: {example_tokens}")

example of tokenized text: ['03', 'Desember', '2018', '11', '55', '59', 'WIB', 'editor', 'Perdana', 'PERAWATAN', 'naskah', 'kuno', 'berusia', 'ratusan', 'tahun', 'koleksi', 'Rekso', 'Pustoko', 'memerlukan', 'biaya', 'tak', 'sedikit', 'biaya', 'Dirinya', 'tak', 'menyangkal', 'bahwa', 'perawatan', 'untuk', 'buku', 'buku', 'berusia', 'sekitar', '200', 'tahun', 'itu', 'terbilang', 'kurang', 'Besarnya', 'biaya', 'perawatan', 'masih', 'menjadi', 'alasan', 'utama', 'sebagian', 'besar', 'koleksi', 'belum', 'tersentuh', 'perawatan', 'layaknya', 'naskah', 'dan', 'manuskrip', 'kuno', 'di', 'museum', 'museum', 'atau', 'perpustakaan', 'besar', 'lainnya', 'Salah', 'satu', 'perawatan', 'yang', 'masih', 'belum', 'baik', 'adalah', 'fumigasi', 'Hingga', 'saat', 'ini', 'perawatan', 'untuk', 'membunuh', 'biota', 'yang', 'merusak', 'arsip', 'hanya', 'bisa', 'dilakukan', 'setahun', 'sekali', 'untuk', 'satu', 'ruang', 'Idealnya', 'fumigasi', 'dilakukan', 'dua', 'kali', 'dalam', 'setahun', 'Selain', 'fumigasi

### 1.2: Lemmatization

Lematisasi merupakan proses untuk mencari bentuk paling dasar dari suatu token dengan memanfaatkan dictionary / mapping dari varian kata (inflected form) menjadi bentuk dasar yang disebut lemma.

Contoh bahasa Indonesia: "menyapu", "disapu", "tersapu" -> "sapu"

Contoh bahasa Inggris: "studies", "studying" -> "study"

Dalam contoh code ini, akan digunakan library [Stanza](https://stanfordnlp.github.io/stanza/), sebuah library natural language processing (NLP) dari Stanford University NLP Group.

Referensi:
- https://stanfordnlp.github.io/stanza/lemma.html
- https://stanfordnlp.github.io/stanza/tokenize.html#start-with-pretokenized-text

In [8]:
# Lemmatize
from stanza.models.common.doc import Document
from stanza.pipeline.core import Pipeline
nlp_stanza: Pipeline = stanza.Pipeline(
  'id',
  processors = 'tokenize,mwt,lemma',
  tokenize_pretokenized = True,
)

def lemmatize_tokens_id(tokens: List[str], nlp_stanza: Pipeline) -> List[str]:
  doc: Document = nlp_stanza([tokens])

  tokens_lemmatized: List[str] = [
    word.lemma for sent in doc.sentences for word in sent.words
  ]
  tokens_lemmatized_without_empty_string: List[str] = [
    token
    for token in tokens_lemmatized
    if (token != None) and (token != '')
  ]

  return tokens_lemmatized_without_empty_string

example_tokens_lemmatized: List[str] = lemmatize_tokens_id(
  tokens = example_tokens,
  nlp_stanza = nlp_stanza,
)
print(f"example of tokens before lemmatization: {example_tokens}")
print(f"example of tokens after lemmatization:  {example_tokens_lemmatized}")

2023-09-12 21:22:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-09-12 21:22:52 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| lemma     | gsd     |

2023-09-12 21:22:52 INFO: Using device: cpu
2023-09-12 21:22:52 INFO: Loading: tokenize
2023-09-12 21:22:52 INFO: Loading: mwt
2023-09-12 21:22:52 INFO: Loading: lemma
2023-09-12 21:22:52 INFO: Done loading processors!


example of tokens before lemmatization: ['03', 'Desember', '2018', '11', '55', '59', 'WIB', 'editor', 'Perdana', 'PERAWATAN', 'naskah', 'kuno', 'berusia', 'ratusan', 'tahun', 'koleksi', 'Rekso', 'Pustoko', 'memerlukan', 'biaya', 'tak', 'sedikit', 'biaya', 'Dirinya', 'tak', 'menyangkal', 'bahwa', 'perawatan', 'untuk', 'buku', 'buku', 'berusia', 'sekitar', '200', 'tahun', 'itu', 'terbilang', 'kurang', 'Besarnya', 'biaya', 'perawatan', 'masih', 'menjadi', 'alasan', 'utama', 'sebagian', 'besar', 'koleksi', 'belum', 'tersentuh', 'perawatan', 'layaknya', 'naskah', 'dan', 'manuskrip', 'kuno', 'di', 'museum', 'museum', 'atau', 'perpustakaan', 'besar', 'lainnya', 'Salah', 'satu', 'perawatan', 'yang', 'masih', 'belum', 'baik', 'adalah', 'fumigasi', 'Hingga', 'saat', 'ini', 'perawatan', 'untuk', 'membunuh', 'biota', 'yang', 'merusak', 'arsip', 'hanya', 'bisa', 'dilakukan', 'setahun', 'sekali', 'untuk', 'satu', 'ruang', 'Idealnya', 'fumigasi', 'dilakukan', 'dua', 'kali', 'dalam', 'setahun', 'Selai

### 1.3: Stemming

Stemming, mirip dengan lematisasi, merupakan aktivitas transformasi kata dari bentuk varian atau *inflected* menjadi bentuk dasar. Bedanya, stemming menggunakan metode algorithmic, yaitu sekumpulan rules untuk memotong karakter (imbuhan) dalam token hingga mencapai bentuk dasar yang disebut stem.

Contoh Bahasa Indonesia: "memakan", "dimakan", "termakan" -> "makan"

Contoh Bahasa Inggris: "wait", "waiting", "waits" -> "wait"

Dalam contoh ini, stemming dilakukan menggunakan tools dari [PySastrawi](https://github.com/har07/PySastrawi).

In [9]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Stemmer.CachedStemmer import CachedStemmer
factory: StemmerFactory = StemmerFactory()
stemmer: CachedStemmer = factory.create_stemmer()

def stem_tokens_id(tokens: List[str], stemmer: CachedStemmer) -> List[str]:
  stemmed_tokens: List[str] = [
      stemmer.stem(token) if token else ''
      for token in tokens
  ]
  stemmed_tokens_without_empty_string: List[str] = [
      token
      for token in stemmed_tokens
      if not ((token == '') or (token == None))
  ]
  return stemmed_tokens_without_empty_string

example_tokens_after_stemming: List[str] = stem_tokens_id(
  tokens = example_tokens_lemmatized,
  stemmer = stemmer,
)
print(f"example of tokens before stemming: {example_tokens_lemmatized}")
print(f"example of tokens after stemming:  {example_tokens_after_stemming}")

example of tokens before stemming: ['03', 'desember', '2018', '11', '55', '59', 'wib', 'editor', 'perdana', 'rawat', 'naskah', 'kuno', 'usia', 'ratus', 'tahun', 'koleksi', 'rekso', 'pustoko', 'perlu', 'biaya', 'tak', 'sedikit', 'biaya', 'dirinya', 'tak', 'sangkal', 'bahwa', 'rawat', 'untuk', 'buku', 'buku', 'usia', 'sekitar', '200', 'tahun', 'itu', 'bilang', 'kurang', 'besar', 'biaya', 'rawat', 'masih', 'jadi', 'alasan', 'utama', 'sebagian', 'besar', 'koleksi', 'belum', 'tersentuh', 'rawat', 'layak', 'naskah', 'dan', 'manuskrip', 'kuno', 'di', 'museum', 'museum', 'atau', 'perpustakaan', 'besar', 'lainnya', 'salah', 'satu', 'rawat', 'yang', 'masih', 'belum', 'baik', 'adalah', 'fumigasi', 'hingga', 'saat', 'ini', 'rawat', 'untuk', 'bunuh', 'biota', 'yang', 'rusak', 'arsip', 'hanya', 'bisa', 'laku', 'setahun', 'sekali', 'untuk', 'satu', 'ruang', 'idealnya', 'fumigasi', 'laku', 'dua', 'kali', 'dalam', 'setahun', 'selain', 'fumigasi', 'untuk', 'jaga', 'tahan', 'kertas', 'agar', 'tidak', 'di

### 1.4: Stop Words Removal

Stop word merupakan kata-kata yang pada umumnya memiliki frekuensi yang sangat tinggi dalam teks namun tidak memberikan informasi yang signifikan.

Contoh stop words bahasa Indonesia: "yang", "di", "pada"

Cotoh stop words bahasa Inggris: "of", "in", "on"

Dalam contoh berikut, kita bisa mendapatkan kumpulan stop words dari tools PySastrawi dan mengunakannya untuk membersihkan token-token yang akan diproses.

In [10]:
# Stop Words Removal
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
stop_words_list: List[str] = stop_factory.get_stop_words()
stop_words_set: Set[str] = set(stop_words_list)

def remove_stop_words_id(tokens: List[str], stop_words: Dict[str, Any]) -> List[str]:
  tokens_without_stop_words: List[str] = [
      token
      for token in tokens
      if token not in stop_words
  ]
  return tokens_without_stop_words

example_tokens_without_stop_words: List[str] = remove_stop_words_id(
  tokens = example_tokens_after_stemming,
  stop_words = stop_words_set,
)

print(f"stop words from PySastrawi: {stop_words_set}\n")
print(f"example of tokens with stop words (before):   {example_tokens_after_stemming}")
print(f"example of tokens without stop words (after): {example_tokens_without_stop_words}")

stop words from PySastrawi: {'dijelaskan', 'enggak', 'hingga', 'menjelaskan', 'berlalu', 'diperbuat', 'tiba', 'keadaan', 'yakin', 'diperlukannya', 'tegas', 'p', 'dahulu', 'ungkapnya', 'tepat', 'ditunjukkannya', 'lah', 'beginilah', 'dulu', 'ini', 'salam', 'masalahnya', 'bagian', 'jauh', 'dilalui', 'yang', 'ternyata', 'tentunya', 'sinilah', 'tersebut', 'terdiri', 'apatah', 'meski', 'per', 'bapak', 'sedangkan', 'kenapa', 'didapat', 'belakangan', 'dirinya', 'selamanya', 'inginkah', 'mengetahui', 'tandasnya', 'lainnya', 'dikerjakan', 'antar', 'bagainamakah', 'depan', 'keterlaluan', 'berawal', 'jelaslah', 'amatlah', 'tampak', 'bersama-sama', 'sebab', 'sepantasnya', 'pihak', 'itu', 'mungkin', 'ketika', 'sebagaimana', 'sendiri', 'semampunya', 'antara', 'sekiranya', 'atau', 'sementara', 'setidak-tidaknya', 'beginikah', 'kapanpun', 'lama', 'merekalah', 'selama-lamanya', 'bung', 'semuanya', 'terhadap', 'dikatakannya', 'memperlihatkan', 'sebaik-baiknya', 'bagaikan', 'semata-mata', 'tempat', 'benar

### 1.5: Preprocess Data

Function-function diatas (tokenisasi, lematisasi, *stemming*, *stop words removal*) dapat dirangkai untuk diterapkan terhadap seluruh data dalam dataset.

In [13]:
# Define your preprocessing pipeline as a function
def preprocess_text_into_tokens_id(text: str,
                                   tokenizer_pattern: str,
                                   nlp_stanza,
                                   stemmer: CachedStemmer,
                                   stop_words: Dict[str, Any]) -> List[str]:
  tokens: List[str] = tokenize_text_id(
    text = text,
    tokenizer_pattern = tokenizer_pattern,
  )
  tokens: List[str] = lemmatize_tokens_id(
    tokens = tokens,
    nlp_stanza = nlp_stanza,
  )
  tokens: List[str] = stem_tokens_id(
    tokens = tokens,
    stemmer = stemmer,
  )
  tokens: List[str] = remove_stop_words_id(
    tokens = tokens,
    stop_words = stop_words,
  )
  return tokens

Untuk menerapkan pipeline pemrosesan teks tersebut kepada seluruh artikel dalam dataset, kita dapat memanfaatkan built-in function map() dalam HuggingFace dataset.

In [19]:
# Apply preprocess to all data
dataset_preprocessed_id: Dataset = dataset_id.map(
  lambda row: dict(
    tokens = preprocess_text_into_tokens_id(
      text = row['text'],
      tokenizer_pattern = tokenizer_pattern,
      nlp_stanza = nlp_stanza,
      stemmer = stemmer,
      stop_words = stop_words_set,
    ),
  ),
  num_proc = 1,
  remove_columns = ['text'],
)
print(f"preprocessed dataset: {dataset_preprocessed_id}")

Map:   0%|          | 0/803 [00:00<?, ? examples/s]

preprocessed dataset: Dataset({
    features: ['tokens'],
    num_rows: 803
})


Hasil akhir dari penerapan tersebut adalah data (tokens) yang siap digunakan untuk pemrosesan selanjutnya dalam aplikasi IR.

In [20]:
print(f"sample of original text (before): {dataset_id[0]['text']}")
print('------')
print(f"sample of preprocessed tokens (after): {dataset_preprocessed_id[0]['tokens']}")

sample of original text (before): 03 Desember 2018, 11: 55: 59 WIB | editor : Perdana
PERAWATAN naskah kuno berusia ratusan tahun koleksi Rekso Pustoko memerlukan biaya tak sedikit biaya. Dirinya tak menyangkal bahwa perawatan untuk buku-buku berusia sekitar 200 tahun itu terbilang kurang. Besarnya biaya perawatan masih menjadi alasan utama sebagian besar koleksi belum tersentuh perawatan layaknya naskah dan manuskrip kuno di museum-museum atau perpustakaan besar lainnya.
Salah satu perawatan yang masih belum baik adalah fumigasi. Hingga saat ini, perawatan untuk membunuh biota yang merusak arsip hanya bisa dilakukan setahun sekali untuk satu ruang. Idealnya fumigasi dilakukan dua kali dalam setahun.
Selain fumigasi, untuk menjaga ketahanan kertas agar tidak dimakan ngengat bisa dilakukan dengan melakukan enkapsulasi dan alih media. Biaya enkapsulasi sendiri cukup besar. Satu lembar enkapsulasi dihargai sebesar Rp 20 ribu. Dalam sebulan ekapsulasi bisa dilakukan pada 1.000 lembar manus

In [21]:
end_time_id: int = time.time()
duration_sec_id: int = end_time_id - start_time_id
duration_min_id: float = round(duration_sec_id / 60, 2)
print(f"Bagian 1: done in {duration_min_id} minutes")

Bagian 1: done in 42.18 minutes


## Bagian 2: Text Preprocessing Bahasa Inggris

Dalam bagian ini, anda diminta untuk melakukan eksplorasi terhadap pemrosesan awal teks seperti pada bagian 1 namun diterapkan ke data dalam bahasa Inggris.

Anda diperbolehkan menambah *code cell* baru sesuai kebutuhan.

In [27]:
start_time_en: int = time.time()
time_struct = time.localtime(start_time_en)
formatted_time = time.strftime("%A, %d %B %Y, %H:%M:%S", time_struct)
print(f"Bagian 2: started at {formatted_time}")

Bagian 2: started at Tuesday, 12 September 2023, 22:05:28


### Load Data

Dalam tugas eksplorasi ini, anda akan menggunakan data corpus dari Common Crawl untuk bahasa Inggris dalam domain spesifik artikel-artikel berita. Code berikut diberikan untuk melakukan load data tersebut.

Referensi:
- https://huggingface.co/datasets/cc_news

In [28]:
dataset_en: Dataset = load_dataset(
    'cc_news',
    split = "train[:3%]",
    num_proc = min(8 * multiprocessing.cpu_count(), 60)
)

Setting num_proc from 60 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/708241 [00:00<?, ? examples/s]

Dataset cc_news downloaded and prepared to C:/Users/USER/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/e3d5612f02fe5f11826a0d9614328b1772e27e5d685f4ec438e7f768e4581734. Subsequent calls will reuse this data.


In [29]:
example_passage_en: str = dataset_en[0]['text']
print(f"example english passage: {example_passage_en}")

example english passage: There's a surprising twist to Regina Willoughby's last season with Columbia City Ballet: It's also her 18-year-old daughter Melina's first season with the company. Regina, 40, will retire from the stage in March, just as her daughter starts her own career as a trainee. But for this one season, they're sharing the stage together.
Performing Side-By-Side In The Nutcracker
Regina and Melina are not only dancing in the same Nutcracker this month, they're onstage at the same time: Regina is doing Snow Queen, while Melina is in the snow corps, and they're both in the Arabian divertissement. "It's very surreal to be dancing it together," says Regina. "I don't know that I ever thought Melina would take ballet this far."
Left: Regina and Melina with another company member post-snow scene in 2003. Right: The pair post-snow scene in 2017 (in the same theater)
Keep reading at dancemagazine.com.


### Preparasi

Anda diperbolehkan untuk melakukan instalasi packages dan download resources tambahan bila diperlukan.

In [35]:
# Install packages as needed
!pip install datasets nltk spacy
!pip install typing-extensions --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [38]:
import spacy
import nltk

ImportError: cannot import name 'deprecated' from 'typing_extensions' (C:\ProgramData\anaconda3\lib\site-packages\typing_extensions.py)

### 2.1: Tokenization

Anda diminta melakukan tokenisasi text menjadi sekumpulan token sama seperti contoh pada Bagian 1.1 dengan ketentuan harus menggunakan regex. Tampilkan penerapan tokenisasi ini kepada contoh salah satu artikel (`example_passage_en`).

In [ ]:
# Tokenize



### 2.2: Lemmatization

Anda diminta melakukan lematisasi untuk token-token yang sudah diproses sebelumnya dengan ketentuan harus menggunakan library [Spacy](https://spacy.io/). Tampilkan penerapan lematisasi ini terhadap hasil tokenisasi kepada contoh sebelumnya. Silahkan lakukan eksplorasi terhadap cara penggunaan tools ini, seperti pembentukan instance Doc, pengambilan lemma dari kata, dsb.

Tips:
- Anda dapat membentuk instance Doc (document) tanpa menggunakan tokenizer dari Spacy.
- Anda dapat melakukan deaktivasi terhadap modul-modul diluar lemmatizer yang tidak dibutuhkan. Hal ini penting untuk peningkatan performa durasi eksekusi.

In [ ]:
# Lemmatize

### 2.3: Stemming

Anda diminta untuk melakukan stemming terhadap kumpulan token dari pemrosesan sebelumnya dengan ketentuan harus menggunakan library [NLTK](https://www.nltk.org/). NLTK memiliki beberapa jenis stemmer untuk bahasa Inggris, dan anda boleh memilih salah satunya. Tampilkan penerapan stemming ini terhadap hasil lematisasi pada contoh sebelumnya.

In [ ]:
# Stemming

### 2.4: Stop Words

Anda diminta untuk mencari kumpulan stop words yang telah disediakan oleh library [NLTK](https://www.nltk.org/). Silahkan lakukan eksplorasi bagaimana cara mendapatkan kumpulan stop words tersebut dan tampilkan ke output untuk anda pelajari. Anda juga diminta untuk mengimplementasikan penghapusan stop words dari token-token yang telah diproses sebelumnya menggunakan kumpulan stop words tersebut.

In [ ]:
# Stop Words Removal

### 2.5: Preprocess Data

Menggunakan function-function yang telah anda buat dari eksplorasi diatas, silahkan rangkai function tokenisasi, lematisasi, dan stemming untuk diterapkan terhadap seluruh data dalam dataset bahasa Inggris yang sudah disediakan, serupa seperti Bagian 1. Khusus untuk tahap ini, anda tidak perlu melakukan stop words removal sebagai persiapan untuk soal selanjutnya.

In [ ]:
# Define your preprocessing pipeline as a function
# Apply preprocess to all data

### 2.6: Word Count

Anda diminta untuk mengumpulkan seluruh token unik beserta dengan frekuensi kemunculannya dalam dataset ini, atau dengan kata lain anda diminta melakukan word counting. Hasil akhir yang diharapkan berbentuk list of tuple (string token, integer count).

Sebagai contoh, bila terdapat artikel A, B, dan C dalam dataset, dimana masing-masing dokumen berisi:
- Artikel A: "computer", "science"
- Artikel B: "computer", "program"
- Artikel C: "program", "execution"

Maka, diharapkan output list of tuple (string, integer) berupa:
- (computer, 2)
- (program, 2)
- (science, 1)
- (execution, 1)

Tips:
- Anda dapat memanfaatkan fungsi map() dari dataset untuk menerapkan function kepada masing-masing row / artikel secara paralel.
- Perhatikan struktur data yang anda gunakan sebagai output dari map(). Struktur data kompleks seperti dictionary dapat memperlambat pemrosesan. Anda dapat melakukan serialization (e.g. menjadi JSON string) bila menemukan isu terkait hal ini.

In [ ]:
# Word Count

### 2.7: Gathering Your Own Stop Words

Setelah anda mendapatkan kumpulan token-frekuensi dari proses sebelumnya, gunakan data tersebut untuk membentuk stop words anda sendiri. Ambil top 200 token yang memiliki kemunculan terbanyak.

Kemudian, tampilkan stop words yang sudah anda kumpulkan dengan stop words dari NLTK. Bandingkan keduanya. Apa yang anda temukan dari komparasi tersebut? Token jenis seperti apa yang hanya ditemukan dari hasil pengumpulan anda?

In [ ]:
# Take top 200 as Stop Words

### 2.8: Character Count

Selanjutnya, mirip dengan word counting, anda diminta untuk mengumpulkan karakter unik dari seluruh token-token dalam dataset beserta dengan frekuensi kemunculannya (character counting).

Sebagai contoh, bila terdapat artikel A dan B dalam dataset, dimana masing-masing dokumen berisi:
- Artikel A: "computer", "science"
- Artikel B: "computer", "program"

Maka, diharapkan output list of tuple (string, integer) berupa:
- ('c', 4)
- ('e', 4)
- ('r', 4)
- ...

Ketentuan:
- Untuk simplifikasi, anda hanya perlu menghitung karakter alphabet (a-z). Silahkan hapus karakter lain (e.g. tanda baca)
- Untuk simplifikasi, anda dapat mengubah huruf kapital dalam seluruh token menjadi huruf kecil.

In [ ]:
# Character Count

### 2.9: Comparing Character Count between English and Indonesian

Kumpulkan juga character count untuk dataset bahasa Indonesia pada Bagian 1. Tampilkan kedua data tersebut dan bandingkan. Apa karakter yang paling dominan dalam bahasa Indonesia? Apa karakter yang paling dominan dalam bahasa Inggris?

In [ ]:
# Character-count for English
# Compare character count between English & Indonesian

In [ ]:
end_time_en: int = time.time()
duration_sec_en: int = end_time_en - start_time_en
duration_min_en: float = round(duration_sec_en / 60, 2)
! date
print(f"Bagian 2: done in {duration_min_en} minutes")

Demikian tugas eksplorasi pemrograman 1. Untuk pengumpulan tugas, silahkan mengikuti petunjuk dalam dokumen soal.